In [ ]:
from datetime import datetime

In [ ]:
from os import scandir

In [ ]:
import triangles as tr

In [ ]:
import csvfile as csv

In [ ]:
import profits as pr

In [ ]:
import csvfile
import time

In [ ]:
from binance.client import Client

api_key = "ucqMzmbqswWPqo4bfQp4DoPyUTFIV2wO749sXmAPyIqwkA4vKQKSO6CZKAPi3DEA"
api_secret = "KRQYhVnepPwutIprQmxyDbugCGCNMkr7WL3Cm7zqQoK9GIuGdLJQpoJqIooTqWnY"

client = Client(api_key, api_secret)

In [ ]:
# This master list collects dictionaries - pair symbols with base and quote assets
my_pairs = []

In [ ]:
def load_pairs():

    ### STEP 1: Load all Binance pairs

    all_tickers = client.get_all_tickers()

    for info in all_tickers:
        my_pairs.append(dict(symbol=info['symbol']))
        
    print(len(my_pairs), 'Binance pairs found and loaded.')

In [ ]:
def update_pairs():

    ### STEP 2: for each pair call Binance again for quote+asset base and status, and add to my_pairs
    ##Risk here - breaching 1200 api calls per minute.....
    ##Risk here - some pairs return None type - ask Binance why :)

    lag = 0.1 # lag for not hitting Binance API 1200/min threshold
    
    print('Loading base and quote currencies, and statuses...')
    for i in range(len(my_pairs)):
        time.sleep(lag)
        details = client.get_symbol_info(my_pairs[i]['symbol'])
        if details is not None:
            my_pairs[i]['baseAsset'] = details['baseAsset']
            my_pairs[i]['quoteAsset'] = details['quoteAsset']
            my_pairs[i]['status'] = details['status']
            if i % 500 == 0: print(time.ctime(),'|| Loaded:',i)
    print('Finished loading base and quote assets, and status.')
    csvfile.lod_to_csv(my_pairs,'my_pairs.csv')

In [ ]:
def get_file_info(name):
    dir_entries = scandir('./')
    for entry in dir_entries:
        if entry.is_file() and entry.name==name:
            info = entry.stat()
    return info.st_mtime

In [ ]:
def convert_date(timestamp):
    d = datetime.utcfromtimestamp(timestamp)
    formated_date = d.strftime('%d %b %Y')
    return formated_date

In [ ]:
# Check last update time for 'my_pairs.csv', 'my_triangles.csv' and 'my_profits.csv'

print('**************************************')
print(f'my_pairs.csv\t\t was last modified: {convert_date(get_file_info("my_pairs.csv"))}')
print(f'my_triangles.csv\t was last modified: {convert_date(get_file_info("my_triangles.csv"))}')
print(f'my_profits.csv\t\t was last modified: {convert_date(get_file_info("my_profits.csv"))}')

In [ ]:
# Interface - are updates needed?

#choice = input("\nDo you want to update my_pairs (~21 mins!) y/n ?")
#if choice=="y": pa.load_pairs(); pa.update_pairs()

#choice = input("\nDo you want to update my_triangles (~4 mins!) y/n ?")
#if choice=="y": tr.find_triangles()

In [ ]:
print("\nBest potential trades")
print("**********************************")

all_profits = pr.find_profits()
buy_buy_sell = pr.sort_profits(all_profits, 6)
buy_sell_sell = pr.sort_profits(all_profits, 11)

print("\nTop 5 for Buy-Buy-Sell")
for i in range(5):
    print(buy_buy_sell[i])

print("\nTop 5 for Buy-Sell-Sell")
for i in range(5):
    print(buy_sell_sell[i])